# 基于AlexNet的试纸识别模型

    程思衍 siyanc@usc.edu

本模型输入的训练数据为试纸图片及其对应属性（阴性、阳性及隐性），在训练过后可以识别试纸属性。

模型上应用深度学习** 卷积神经网络 **架构，基于Hinton及其学生Alex Krizhevsky[1]在2012年ImageNet竞赛中所夺魁的AlexNet构建训练架构，以达到对小范围内的物体进行卷积识别。

我们将逐步介绍代码结构。

## 1.引入所需要的包并将训练环境设为GPU环境

本模型应用像素大小为（224，224）的黑白图片作为输入，经过多层卷积池化，故参数级别为1e+10，建议采用云或本地GPU环境进行训练。

我们首先如下导入所需要的包/库，并将keras环境设置为GPU环境。

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot
from sklearn.cross_validation import train_test_split

from keras import backend as K
from keras import optimizers
from keras import metrics
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.initializers import he_normal
from keras.losses import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
K.set_image_dim_ordering('th')

#setting GPU
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} )
sess = tf.Session(config=config)
K.set_session(sess)
print ('Using GPU for Computing')

## 2.读取图片

随后我们读取图片作为输入数据。

训练数据应当放置在源代码同一路径下，并分别命名为0，1，2。

在0文件夹中放置显隐性的试纸图片，在1文件夹中放置阴性（一条杠）试纸图片，在2文件夹中放置显阳性（两条杠）的试纸图片。

读取同时我们将图片转换为（224，224）像素的统一大小并统一转换为黑白图片以便训练。

In [ ]:
#read images from folders
images = []
#labels = [] 
images_stat = []

##############################################################
#here we have 3 labels. The folders are named as 0/1/2, images
#that have the same labels locate in same folder.
#We read images and their labels below and resize them to 
#(224, 224) for future computation.
##############################################################
for folder in ['./0/','./1/','./2/']:
    images_stat.append(len(os.listdir(folder)))
    for image_name in os.listdir(folder):
        image = cv2.imread(os.path.join(folder,image_name), cv2.IMREAD_GRAYSCALE)
        if image is not None:
            images.append(cv2.resize(image, (224, 224),interpolation=cv2.INTER_NEAREST))
            #labels.append(folder.split('/')[1])

#Plot one of the images
images = np.array(images).reshape(images.shape[0], 224, 224)
plt.imshow(images[15], cmap='gray')

## 3.分离数据并生成标签

按照上面代码的顺序，我们首先读入的是0文件夹中的文件，然后是1中的文件，最后才是2中文件。

images_stat的作用的便是记录各个文件夹中包含多少个文件，并以此为依据分离数据并生成标签。

In [ ]:
#Reshapa data and generate their label
images = np.array(images).reshape(images.shape[0], 1, 224, 224)
data0 = images[0:images_stat[0]]
label0 = np.zeros((data0.shape[0],))

data1 = images[images_stat[0]:images_stat[0]+images_stat[1]]
label1 = np.zeros((data1.shape[0],))+1

data2 = images[images_stat[0]+images_stat[1]:]
label2 = np.zeros((data2.shape[0],))+2

images.astype('float32')

## 4. 扩张训练数据

我们应用keras内建的ImageDataGernertor对训练数据进行扩张。

在本报告撰写的同时，本模型只有少量数据可供训练，为保证模型鲁棒性，在这里将数据规模扩张。若后续能够有更多的数据，则可以考虑删去该部分或是将变换系数减小。

该接口会随机将图片进行旋转、上下平移、左右平移、放大、镜像放置。

在这边我们将每张图片变换为500张，由底下的iteration_time声明，并将图片分别保存至auged_0/auged_1/auged_2文件夹中以供查看。

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        cval=0,
        vertical_flip=False)
# fit parameters from data
datagen.fit(images)
# configure batch size and retrieve one batch of images
iteration_time = 500

os.makedirs('auged_0')
data0_aug = np.array([])

i = 0
for X_batch, y_batch in datagen.flow(data0, label0, batch_size=data0.shape[0], save_to_dir='auged_0', save_prefix='aug', save_format='png'):
    print (i)
    if i >= iteration_time:
        break
    if len(data0_aug) == 0:
        data0_aug = X_batch
    else:
        data0_aug = np.vstack((data0_aug, X_batch))
    i = i + 1

In [ ]:
os.makedirs('auged_1')
data1_aug = np.array([])

i = 0
for X_batch, y_batch in datagen.flow(data1, label1, batch_size=data1.shape[0], save_to_dir='auged_1', save_prefix='aug', save_format='png'):
    print (i)
    if i >= iteration_time:
        break
    if len(data1_aug) == 0:
        data1_aug = X_batch
    else:
        data1_aug = np.vstack((data1_aug, X_batch))
    i = i + 1

In [ ]:
os.makedirs('auged_2')
data2_aug = np.array([])

i = 0
for X_batch, y_batch in datagen.flow(data2, label2, batch_size=data2.shape[0], save_to_dir='auged_2', save_prefix='aug', save_format='png'):
    print(i)
    if i >= iteration_time:
        break
    if len(data2_aug) == 0:
        data2_aug = X_batch
    else:
        data2_aug = np.vstack((data2_aug, X_batch))
    i = i + 1

## 5.数据综合预处理

至此，我们已拥有足量的训练数据及标签，在本代码块中我们首先拼接之前逐类扩张过的数据。

随后将训练数据保存至data.npy文件中，研究人员可以随时读取以加快效率。

紧接着我们将数据随机打乱以求客观性。

最后我们将这些数据分为训练数据和测试数据，训练数据占75%，测试数据占25%。

In [ ]:
NUMBER_OF_CLASSES = 3

#Concatenate data and labels
auged_labels = np.vstack((np.zeros((data0.shape[0]*(iteration_time+1),1)), np.zeros((data1.shape[0]*(iteration_time+1),1))+1,\
                          np.zeros((data2.shape[0]*(iteration_time+1),1))+2))
auged_labels = auged_labels.reshape((auged_labels.shape[0],))
auged_data = np.vstack((data0, data0_aug, data1, data1_aug,data2, data2_aug))

#save data, so you can resume and load data from here to continue the experiment.
np.save('data.npy',auged_data)

#shuffle data for objectivity
rd_idx = [i for i in range(len(auged_data))]
np.random.shuffle(rd_idx)

shuffled_data = auged_data[rd_idx]
shuffled_labels = auged_labels[rd_idx]

#normalize data
normalized_data = shuffled_data/255

#train and test split for 0.75-0.25
X_train, X_test, y_train, y_test = train_test_split(normalized_data, shuffled_labels, test_size=0.25, random_state=42)

print (X_train.shape)
print (X_test.shape)

#make labels categorical
Y_train = np_utils.to_categorical(y_train, NUMBER_OF_CLASSES)
Y_test = np_utils.to_categorical(y_test, NUMBER_OF_CLASSES)

## 6. 模型具体架构

![image](Arch.png)

模型应用Keras贯序接口构建，详细信息请查询[Keras API](http://keras-cn.readthedocs.io/en/latest/)。

模型架构为：

> * 拥有96个卷积核，卷积核大小为（11，11），步长为4，激活函数为“relu”的卷积层
> * 池化窗口为（3，3），步长为（2，2）的最大池化层

> * 拥有96个卷积核，卷积核大小为（11，11），步长为4，激活函数为“relu”的卷积层
> * 池化窗口为（3，3），步长为（2，2）的最大池化层

> * 拥有384个卷积核，卷积核大小为（3，3），步长为1，激活函数为“relu”的卷积层
> * 拥有384个卷积核，卷积核大小为（3，3），步长为1，激活函数为“relu”的卷积层
> * 拥有256个卷积核，卷积核大小为（3，3），步长为1，激活函数为“relu”的卷积层
> * 池化窗口为（3，3），步长为（2，2）的最大池化层

> * 拥有4096个神经元，激活函数为“relu”的全连接层
> * 拥有4096个神经元，激活函数为“relu”的全连接层
> * 拥有3个神经元，激活函数为“softmax”的输出层

该卷积神经网络架构在2012年ImageNet比赛中成名，并带动整个深度学习领域的发展，这边是我们在该问题中使用AlexNet的原因。

In [ ]:
## Training
model = Sequential()

## Architecture
model.add(Conv2D( filters = 96, kernel_size = (11,11), strides = 4, padding = 'same', activation = 'relu', input_shape = (1, 224, 224), kernel_initializer = 'he_normal'))
model.add(MaxPooling2D( pool_size = (3,3), strides = (2,2), padding= 'same', data_format = None)) # overlapping pooling

model.add(Conv2D( filters = 256, kernel_size = (5,5), strides = 1, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'))
model.add(MaxPooling2D( pool_size = (3,3), strides = (2,2), padding= 'same', data_format = None)) 

model.add(Conv2D( filters = 384, kernel_size = (3,3), strides = 1, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'))
model.add(Conv2D( filters = 384, kernel_size = (3,3), strides = 1, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'))
model.add(Conv2D( filters = 256, kernel_size = (3,3), strides = 1, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal'))
model.add(MaxPooling2D( pool_size = (3,3), strides = (2,2), padding= 'same', data_format = None))

model.add(Flatten())
model.add(Dense( units = 4096, activation = 'relu'))
model.add(Dense( units = 4096, activation = 'relu'))

model.add(Dense( units = 3, activation = 'softmax'))
model.summary()

## Optimizer
adam = optimizers.adam(lr=0.00005, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay= 0, amsgrad = False)
momentum = optimizers.SGD(lr=0.01, momentum = 0.9, decay=1e-6)

In [ ]:
#Compile and trian
model.compile(optimizer = adam, loss = categorical_crossentropy, metrics=[metrics.categorical_accuracy])
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data=(X_test, Y_test))